In [1]:
from langgraph.graph import StateGraph, START, END
from ads4gpts_langchain import Ads4GPTsToolkit
from langgraph.prebuilt import ToolNode
from utils.nodes import (
    agent_node,
    ad_node,
    clean_up_ad_node,
    agent_edge,
)
from utils.datamodels import State

import os

## Define the Graph

The graph in this example is a standard approach of using ADS4GPTs in a LangGraph application. 

### Agent Node

This can be any node that feeds into the advertising nodes. Here is a simple assistant node.

### Agent Edge

Most often you would want to use conditional logic in order to control the flow of Ads in your application. The agent edge accomplishes that role in this example. The difference with other examples is that the conditional logic is dynamic and changes throughout execution

### Ad Node

We encapsulate the Ad Agent from ADS4GPTs in an Ad Node.

### Ad ToolNode

This is a prebuilt node from LangGraph that executes the Ads4GptsTool.

### Clean Up Nod

We do not wish to introduce Ads into the State messages. The state should only have useful content for the application.

In [ ]:
# Use the default key if you don't have a key already
ADS4GPTS_API_KEY = os.getenv("ADS4GPTS_API_KEY", "default_api_key")

# Define the workflow of the graph
workflow = StateGraph(State)
ad_toolkit = Ads4GPTsToolkit(ads4gpts_api_key=ADS4GPTS_API_KEY)
ad_tool_node = ToolNode(ad_toolkit.get_tools())

workflow.add_node("agent_node", agent_node)
workflow.add_node("ad_node", ad_node)
workflow.add_node("ad_tool_node", ad_tool_node)
workflow.add_node("clean_up_ad_node", clean_up_ad_node)

workflow.add_edge(START, "agent_node")
workflow.add_conditional_edges("agent_node", agent_edge)
workflow.add_edge("ad_node", "ad_tool_node")
workflow.add_edge("ad_tool_node", "clean_up_ad_node")
workflow.add_edge("clean_up_ad_node", END)

graph = workflow.compile()

## Stream the Graph

In [3]:
# Run the graph to get a response and an Ad at the end
inputs = {"messages": ["Hello world!"]}
async for event in graph.astream(inputs):
    for k, v in event.items():
        if k != "__end__":
            print(v)

2024-11-22 13:39:35,726 - ads4gpts_langchain.agent - INFO - ChatOpenAI model initialized successfully.
2024-11-22 13:39:35,726 - ads4gpts_langchain.agent - INFO - ChatOpenAI model initialized successfully.
2024-11-22 13:39:35,727 - ads4gpts_langchain.agent - INFO - Ads4GPTsToolkit initialized and tools retrieved.
2024-11-22 13:39:35,727 - ads4gpts_langchain.agent - INFO - Ads4GPTsToolkit initialized and tools retrieved.
2024-11-22 13:39:35,808 - ads4gpts_langchain.agent - INFO - Ads4GPTs agent created successfully.
2024-11-22 13:39:35,808 - ads4gpts_langchain.agent - INFO - Ads4GPTs agent created successfully.


{'messages': [AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 22, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-ff77ebce-76a3-4f0a-9627-5136753d174b-0', usage_metadata={'input_tokens': 22, 'output_tokens': 9, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})], 'ad_counter': 1}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3t2W3wRQXvkQllbIOK0tNTjk', 'function': {'arguments': '{\n"context": "Hello world!",\n"num_ads": 1\n}', 'name': 'ads4gpts_tool'},